In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.preprocessing import preprocess_input_df
import aequitas.plot as ap


### Set up data

In [5]:
toy_columns = ['StudentId',
# .66 Acc RF
'Sex',
'Race',
'CompositeGrades',
'StandardizedTestQuartile',
'CategoricalGrades',

# .86 Acc RF (with base), .96 (second two), .58 (alone)
'TimeSpentOnHomeworkInSchool',
'TimeSpentOnHomeworkOutOfSchool',
'TimeSpentOnExtracurriculars',
'HelpWithEssays',
'ParentsCheckHomework',

# .92 Acc FR (with base), .82 (alone)
'HighSchoolHelpedWithSchoolApplication',
'HelpedWithFinancialAidApp',
'ParentsHighestLevelEducation',
'Socio-economicStatusQuartile',
'PrivateSchoolExpenses',
'PrivateTutoringExpenses',
'Tutored',
'FathersWishes',
'MothersWishes',

'GradesUndergrad']
target_col = 'GradesUndergrad'

In [6]:
nels = pd.read_csv('data/NELS_Filtered.csv')
nels_columns = pd.read_csv('data/NELS_Subset_Columns.csv')
nels_columns_readable = pd.read_csv('data/NELS_Subset_Columns_Human_Readable.csv')
nels_subset = nels[list(nels_columns.columns)]
nels_subset.columns = list(nels_columns_readable.columns)

In [7]:
# Run this cell to downsample
from sklearn.utils import resample
def resample_up_down(dataframe, upsample=True, target_col=target_col):
    # Separate majority and minority classes
    df_majority = dataframe[dataframe[target_col]==1]
    df_minority = dataframe[dataframe[target_col]==0]
    
    if upsample:
        # Upsample minority class
        df_minority_upsampled = resample(df_minority, 
                                        replace=True,
                                        n_samples=len(df_majority),
                                        random_state=0)
    
        # Combine majority class with upsampled minority class
        df_resampled = pd.concat([df_majority, df_minority_upsampled])
    else:
        # Downsample majority class
        df_majority_downsampled = resample(df_majority, 
                                        replace=False,
                                        n_samples=len(df_minority),
                                        random_state=0) 
        
        # Combine minority class with downsampled majority class
        df_resampled = pd.concat([df_majority_downsampled, df_minority])
        
    # Display new class counts
    print(df_resampled[target_col].value_counts())

    return df_resampled

In [8]:
# Only valid grades
toy_dataframe = nels_subset[(1 <= nels_subset[target_col]) & (nels_subset[target_col] <= 7)]

# Only white/black nonhispanic
toy_dataframe = toy_dataframe[(3 <= toy_dataframe['Race']) & (toy_dataframe['Race'] <= 4)]

# Shuffle randomly before training models
toy_dataframe = toy_dataframe.sample(frac=1).reset_index(drop=True)

toy_dataframe = toy_dataframe[toy_columns]

# Make binary decision >= 2.75 GPA
toy_dataframe[target_col] = np.where(toy_dataframe[target_col] <= 3, 1, 0)
make_string = False

if make_string:
    # Make binary decision >= 2.75 GPA
    toy_dataframe["Race"] = np.where(toy_dataframe["Race"] == 3, "Black", "White")
    toy_dataframe["Sex"] = np.where(toy_dataframe["Sex"] == 1, "Male", "Female")

toy_dataframe = resample_up_down(toy_dataframe, upsample=True, target_col=target_col)

# Withhold columns that are improper for prediction (like having completed PSE)
columns_to_withhold = ["StudentId"]

X = toy_dataframe[toy_dataframe.columns.difference([target_col]+columns_to_withhold)]
y = toy_dataframe[target_col]

1    5154
0    5154
Name: GradesUndergrad, dtype: int64


### Train classifier

In [9]:
def set_cols_zero(df, cols):
    for col in cols:
        df[col] = 0
    return df

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

x_test_with_protected = x_test.copy()
x_train = set_cols_zero(x_train, ['Sex', 'Race'])
x_test = set_cols_zero(x_test, ['Sex', 'Race'])


rf = RandomForestClassifier() # max_iter=1000 
rf.fit(x_train, y_train)

predictions = rf.predict(x_test)

print(classification_report(y_test, predictions)) # target_names=target_names

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      1048
           1       0.97      0.91      0.94      1014

    accuracy                           0.94      2062
   macro avg       0.94      0.94      0.94      2062
weighted avg       0.94      0.94      0.94      2062



### Run Aequitas

In [25]:
def get_sex_string(sex):
    if sex == 1:
        return "Male" 
    elif sex == 2:
        return "Female"
    else:
        return "Not specified"

def get_race_string(race):
    if race == 3:
        return "Black"
    elif race == 4:
        return "White"
    else:
        return "Other"

In [26]:
predictions_df = pd.DataFrame(predictions, columns=['score'])
df = pd.concat([x_test_with_protected, y_test], axis=1).reset_index(drop=True)
df = pd.concat([df, predictions_df], axis=1)
df = df.rename(columns={'GradesUndergrad': 'label_value'})
df = df[['label_value', 'score', 'Sex', 'Race']]


df['Sex'] = df.apply(lambda x: get_sex_string(x.Sex), axis=1)
df['Race'] = df.apply(lambda x: get_race_string(x.Race), axis=1)
df['score'] = df.apply(lambda x: int(x.score), axis=1)

df_train = df.copy()
df_train['Sex']

attributes_and_reference_groups = {'Sex': 'Female', 'Race': 'Black'}
attributes_to_audit = list(attributes_and_reference_groups.keys())


In [27]:
df.to_csv('aequitas_data.csv')

In [28]:
g = Group()
b = Bias()

# get_crosstabs returns a dataframe of the group counts and group value bias metrics.
xtab, _ = g.get_crosstabs(df, attr_cols=attributes_to_audit)
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict=attributes_and_reference_groups)

get_disparity_predefined_group()


### Examine fairness results

In [29]:
f = Fairness()
gvf = f.get_group_value_fairness(bdf)
gaf = f.get_group_attribute_fairness(gvf)
gaf

,model_id,score_threshold,attribute_name,Statistical Parity,Impact Parity,FDR Parity,FPR Parity,FOR Parity,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,0,binary 0/1,Race,False,False,False,False,False,False,True,True,True,True,False,False,False,False,False
1,0,binary 0/1,Sex,False,False,False,False,False,False,True,True,True,True,False,False,False,False,False


In [30]:
metrics = ['tpr']
disparity_tolerance = 1.1

In [31]:
ap.disparity(bdf, metrics, 'Sex', fairness_threshold = disparity_tolerance)

alt.LayerChart(...)

In [32]:
ap.absolute(bdf, metrics, 'Sex', fairness_threshold = disparity_tolerance)

alt.LayerChart(...)

In [33]:
ap.disparity(bdf, metrics, 'Race', fairness_threshold = disparity_tolerance)

alt.LayerChart(...)

In [34]:
ap.absolute(bdf, metrics, 'Race', fairness_threshold = disparity_tolerance)

alt.LayerChart(...)